In [19]:
from tkinter import *
import numpy as np
from tkinter import font
import skfuzzy as fuzz
from skfuzzy import control as ctrl
root = Tk()
root.title("Irrigation")
frame = LabelFrame(root,padx = 50,pady = 50)
frame.pack(padx = 100,pady = 100)
label1 = Label(frame,text = "humidity:",font = "Helvetica 12 bold")
label1.pack()
f = font.Font(label1, label1.cget("font"))
f.configure(underline=True)
label1.configure(font=f)
humidity = Entry(frame,width = 20,bd = 6)
humidity.insert(0,'0')
humidity.pack()
result = Entry(frame,width = 20,bd = 6)
result2 = Entry(frame,width = 20,bd = 6)
label2 = Label(frame,text = "temperature:",font = "Helvetica 12 bold")
label2.pack()
label3 = Label(frame,text = "soil:",font = "Helvetica 12 bold")
f3 = font.Font(label3, label3.cget("font"))
f3.configure(underline=True)
label3.configure(font=f3)
f2 = font.Font(label2, label2.cget("font"))
f2.configure(underline=True)
label2.configure(font=f2)
temperature = Entry(frame,width = 20,bd = 6)
temperature.insert(0,'0')
temperature.pack()
label3.pack()
Soil = Entry(frame,width = 20,bd = 6)
Soil.insert(0,'0')
Soil.pack()
different_methods = ["centroid" , "lom" , "mom" , "som"]
drop_down = StringVar(frame)
drop_down_options = OptionMenu(frame,drop_down,*different_methods)
drop_down_options.config(bg = "BLACK",fg = "PURPLE")
drop_down_options.pack(side='bottom')
#define antecedents
Temperature = ctrl.Antecedent(np.arange(0,46,1),"Temperature")
Humidity = ctrl.Antecedent(np.arange(0,30,1),"Humidity")
soil = ctrl.Antecedent(np.arange(0,46,1),"soil")
    #define the consequents
Duration = ctrl.Consequent(np.arange(0,50,1),"Duration")
Nutriments = ctrl.Consequent(np.arange(0,161,1),"Nutriments")
    #draw the graphs
    #temperature
Temperature["Cold"] = fuzz.trapmf(Temperature.universe,[0,0,5,10])
Temperature["Fresh"] = fuzz.trimf(Temperature.universe,[5,12.5,22.5])
Temperature["Normal"] = fuzz.trimf(Temperature.universe,[15,22.5,27.5])
Temperature["Hot"] = fuzz.trimf(Temperature.universe,[22.5,30,37.5])
Temperature["Very Hot"] = fuzz.trapmf(Temperature.universe,[32.5,37.5,45,45])
#soil
soil["shallow"] = fuzz.trapmf(soil.universe,[0,0,5,10])
soil["normal"] = fuzz.trimf(soil.universe,[5,15,25])
soil["deep"] = fuzz.trapmf(soil.universe,[20,33,50,50])
    #humidity
Humidity["Dry"] = fuzz.trapmf(Humidity.universe,[0,0,5,12.5])
Humidity["Humid"] = fuzz.trimf(Humidity.universe,[5,12.5,22.5])
Humidity["Wet"] = fuzz.trapmf(Humidity.universe,[17.5,20,30,30])
    #Duration
Duration["Short"] = fuzz.trapmf(Duration.universe,[0,0,10,20])
Duration["Medium"] = fuzz.trimf(Duration.universe,[10,30,50])
Duration["Long"] = fuzz.trapmf(Duration.universe,[40,50,70,70])
    #Nutrients
Nutriments["tiny"] = fuzz.trapmf(Nutriments.universe,[0,0,10,20])
Nutriments["small"] = fuzz.trimf(Nutriments.universe,[15,25.5,40])
Nutriments["medium"] = fuzz.trimf(Nutriments.universe,[30,45,70])
Nutriments["large"] = fuzz.trimf(Nutriments.universe,[60,75.5,100])
Nutriments["Very large"] = fuzz.trapmf(Nutriments.universe,[90,110,160,160])
    #now, we define the rules
R1 = ctrl.Rule(Temperature["Very Hot"] & Humidity["Dry"],(Duration["Long"] , Nutriments["tiny"]))
R2 = ctrl.Rule(Temperature["Hot"] & Humidity["Dry"],Duration["Long"] , Nutriments["small"])
R21 = ctrl.Rule(Temperature["Hot"] & Humidity["Dry"] & soil["deep"],(Duration["Long"] , Nutriments["tiny"]))
R3 = ctrl.Rule(Temperature["Fresh"] & Humidity["Dry"],(Duration["Long"] , Nutriments["medium"]))
R31 = ctrl.Rule(Temperature["Fresh"] & Humidity["Dry"] & (soil["deep"] | soil["normal"]),(Duration["Medium"] , Nutriments["small"]))
R4 = ctrl.Rule(Temperature["Normal"] & Humidity["Dry"],(Duration["Long"] , Nutriments["medium"]))
R41 = ctrl.Rule(Temperature["Normal"] & Humidity["Dry"] & soil["normal"],(Duration["Medium"] , Nutriments["medium"]))
R5 = ctrl.Rule(Temperature["Cold"] & Humidity["Dry"] & (soil["deep"] | soil["normal"]),(Duration["Medium"] , Nutriments["large"]))
R6 = ctrl.Rule(Temperature["Very Hot"] & Humidity["Humid"],(Duration["Medium"] , Nutriments["large"]))
R7 = ctrl.Rule(Temperature["Hot"] & Humidity["Humid"],(Duration["Medium"] , Nutriments["medium"]))
R8 = ctrl.Rule(Temperature["Fresh"] & Humidity["Humid"],(Duration["Medium"] , Nutriments["small"]))
R9 = ctrl.Rule(Temperature["Normal"] & Humidity["Humid"],(Duration["Medium"] , Nutriments["medium"]))
R91 = ctrl.Rule((Temperature["Normal"]  | Temperature["Fresh"] | Temperature["Hot"] | Temperature["Very Hot"]) 
                & Humidity["Humid"] & soil["shallow"],(Duration["Long"] , Nutriments["Very large"]))
R92 = ctrl.Rule((Temperature["Normal"]  | Temperature["Fresh"] | Temperature["Hot"] | Temperature["Very Hot"]) 
                & Humidity["Humid"] & soil["deep"],(Duration["Short"] , Nutriments["small"]))
R10 = ctrl.Rule(Temperature["Cold"] & Humidity["Humid"],(Duration["Short"] , Nutriments["tiny"]))
R101 = ctrl.Rule(Temperature["Cold"] & Humidity["Humid"] & soil["shallow"],(Duration["Medium"] , Nutriments["Very large"]))
R11 = ctrl.Rule(Temperature["Very Hot"] & Humidity["Wet"],(Duration["Medium"] , Nutriments["medium"]))
R121 = ctrl.Rule(Temperature["Hot"] & Humidity["Wet"] & soil["deep"],(Duration["Short"] , Nutriments["small"]))

R12 = ctrl.Rule(Temperature["Hot"] & Humidity["Wet"],(Duration["Medium"] , Nutriments["small"]))
R13 = ctrl.Rule(Temperature["Fresh"] & Humidity["Wet"],(Duration["Short"] , Nutriments["medium"]))
R14 = ctrl.Rule(Temperature["Normal"] & Humidity["Wet"],(Duration["Short"] , Nutriments["large"]))
R15 = ctrl.Rule(Temperature["Cold"] & Humidity["Wet"],(Duration["Short"] , Nutriments["large"]))
R16 = ctrl.Rule(Temperature["Cold"] & Humidity["Wet"] & soil["shallow"],(Duration["Short"] , Nutriments["Very large"]))
    #create the control system
irrigating_ctrl = ctrl.ControlSystem([R1,R2,R21,R3,R31,R4,R41,R5,R6,R7,R8,R9,R91,R92,R10,R101,R11,R12,R121,R13,R14,R15,R16])
irrigating = ctrl.ControlSystemSimulation(irrigating_ctrl)
def runFuzzy():
    Duration.defuzzify_method = drop_down.get()
    result.delete(0,10)
    if int(temperature.get()) <= 0 or int(humidity.get()) <= 0 or int(Soil.get()) <= 0:
        return
    irrigating.input["Temperature"] = int(temperature.get())
    irrigating.input["Humidity"] = int(humidity.get())
    irrigating.input["soil"] = int(Soil.get())
    irrigating.compute()
    
    result.insert(0,str(irrigating.output['Duration']))
    result2.insert(0,str(irrigating.output['Nutriments']))


okButton = Button(frame,text = "compute",command = runFuzzy,bd = 5)
okButton.pack()
result_label = Label(frame,text = "irrigation time:",font = "Helvetica 12 bold")
result_label.pack()
f3 = font.Font(result_label, result_label.cget("font"))
f3.configure(underline=True)
result_label.configure(font=f3)
result.pack()
nutrient_label = Label(frame,text = "nutrient uptake:",font = "Helvetica 12 bold")
nutrient_label.pack()
f4 = font.Font(nutrient_label, nutrient_label.cget("font"))
f4.configure(underline=True)
nutrient_label.configure(font = f4)
result2.pack()
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Charbel\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-19-ae1e08d5f9af>", line 111, in runFuzzy
    irrigating.compute()
  File "C:\Users\Charbel\Desktop\courses\lab ai\Fuzzy logic students\skfuzzy\control\controlsystem.py", line 373, in compute
    CrispValueCalculator(consequent, self).defuzz()
  File "C:\Users\Charbel\Desktop\courses\lab ai\Fuzzy logic students\skfuzzy\control\controlsystem.py", line 587, in defuzz
    self.var.defuzzify_method)
  File "C:\Users\Charbel\Desktop\courses\lab ai\Fuzzy logic students\skfuzzy\defuzzify\defuzz.py", line 266, in defuzz
    raise ValueError('The input for `mode`, %s, was incorrect.' % (mode))
ValueError: The input for `mode`, , was incorrect.


In [15]:
R1.label


Nutriments[tiny]